# 03 - Modeling (OLS Regression) & Results
In this notebook, we build a multiple linear regression model to understand how different apartment characteristics — especially construction age (Bauperiode) — influence CHF per m².

We will:

1. Load the cleaned dataset from SQLite
2. Simplify Bauperiode Categories (with explanation)
3. Select baselines 
4. Fit OLS model
5. Interpret the results

## Setup

In [1]:
import pandas as pd
import sqlite3
import statsmodels.api as sm


## 1. Load Cleaned Data from SQLite


In [2]:

conn = sqlite3.connect("../Data/apartment_database.db")
df = pd.read_sql("SELECT * FROM rental_listings_clean", conn)
conn.close()

df.head()



,id,area_m2,rooms,rent_chf,year_built,address,canton,bauperiode,chf_per_m2,m2_per_chf
0,1,134,5.5,2530,1984,"Kesslernmattstr. 14, 8965 Berikon",Zurich,Zwischen 1981 und 1990 erbaut,18.880597,0.052964
1,2,98,3.5,2255,1989,"Im Spitzler 21, 8902 Urdorf",Zurich,Zwischen 1981 und 1990 erbaut,23.010204,0.043459
2,3,68,3.5,1945,1973,"Ferdinand Hodler-Str. 14, 8049 Zürich",Zurich,Zwischen 1971 und 1980 erbaut,28.602941,0.034961
3,4,134,5.5,5590,1906,"Weinbergstrasse 72, 8006 Zürich",Zurich,Vor 1919 erbaut,41.716418,0.023971
4,5,94,4.5,2360,1971,"Bahnhofstr. 73, 8957 Spreitenbach",Zurich,Zwischen 1971 und 1980 erbaut,25.106383,0.039831


## 2. Simplifying Bauperiode Categories
The original BFS dataset contains 12 detailed construction-age categories.
For the purpose of this analysis, we consolidate these into five broader groups.
This decision is based on three methodological considerations:
1. Interpretability
Several original categories differ only by short time spans (e.g. 2001–2005 vs. 2006–2010).
Combining them leads to clearer, more meaningful comparisons between construction eras.

2. Sufficient sample size per category
Some construction periods appear only a few times in the scraped dataset.
Consolidating categories ensures that each group contains enough observations to support reliable analysis.

3. Alignment with common building-age cohorts in Swiss real-estate research
The final grouping reflects typical transitions in building standards, such as pre-war stock, post-war expansion, modernization phases, and recent energy-efficient construction.

We therefore use the following simplified structure:
- very_old : vor 1919, 1919-1945
- old : 1946-1960, 1961-1970
- mid : 1971-1980, 1981-1990
- recent: 1991-2000, 2001-2005, 2006-2010
- new : 2011-2015, 2016-2020, 2021-2024

This grouping produces well-balanced category sizes and clearer insights into how different building eras relate to rental values per m².

In [3]:
def simplify_bauperiode(cat):
    if cat in ["Vor 1919 erbaut", "Zwischen 1919 und 1945 erbaut"]:
        return "very_old"
    if cat in ["Zwischen 1946 und 1960 erbaut", "Zwischen 1961 und 1970 erbaut"]:
        return "old"
    if cat in ["Zwischen 1971 und 1980 erbaut", "Zwischen 1981 und 1990 erbaut"]:
        return "mid"
    if cat in ["Zwischen 1991 und 2000 erbaut", "Zwischen 2001 und 2005 erbaut",
               "Zwischen 2006 und 2010 erbaut"]:
        return "recent"
    if cat in ["Zwischen 2011 und 2015 erbaut", "Zwischen 2016 und 2020 erbaut",
               "Zwischen 2021 und 2024 erbaut"]:
        return "new"
    return "unknown"

df["bauperiode_simplified"] = df["bauperiode"].apply(simplify_bauperiode)
df["bauperiode_simplified"].value_counts()


bauperiode_simplified
old         32
new         29
recent      21
unknown     16
mid         14
very_old     8
Name: count, dtype: int64

## 3. Selecting Baselines
To make the results easy to interpret:

Canton baseline: Zurich

Bauperiode baseline: new (2011–2024)

These represent the newest buildings and the strongest rental market.

In [4]:
predictors = ["area_m2", "bauperiode_simplified", "canton"]

# One-hot encoding without automatic baseline
X = pd.get_dummies(df[predictors], drop_first=False)

# Drop chosen baselines manually
X = X.drop(columns=[
    "canton_Zurich",
    "bauperiode_simplified_new"
], errors="ignore")

# Convert all predictors to numeric (critical for Statsmodels)
X = X.apply(pd.to_numeric, errors="coerce").astype(float)

# Target variable
y = pd.to_numeric(df["chf_per_m2"], errors="coerce").astype(float)

X.head()


,area_m2,bauperiode_simplified_mid,bauperiode_simplified_old,bauperiode_simplified_recent,bauperiode_simplified_unknown,bauperiode_simplified_very_old,canton_Bern,canton_Luzern
0,134.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,98.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,68.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,134.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,94.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4. Fit OLS Model
We fit a multiple linear regression model using Statsmodels

In [5]:
X_const = sm.add_constant(X)
model = sm.OLS(y, X_const).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             chf_per_m2   R-squared:                       0.489
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     13.25
Date:                Mon, 01 Dec 2025   Prob (F-statistic):           2.70e-13
Time:                        17:25:58   Log-Likelihood:                -401.57
No. Observations:                 120   AIC:                             821.1
Df Residuals:                     111   BIC:                             846.2
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             48.4991      3.168     15.311      0.000      42.222      54.776
area_m2                           -0.1090      0.027     -4.005      0.000      -0.163      -0.055
bauperiode_simplified_mid        -10.3110      2.358     -4.373      0.000     -14.983      -5.638
bauperiode_simplified_old         -6.0492      1.878     -3.222      0.002      -9.770      -2.328
bauperiode_simplified_recent      -6.9713      2.064     -3.377      0.001     -11.062      -2.881
bauperiode_simplified_unknown     -2.1391      2.273     -0.941      0.349      -6.643       2.365
bauperiode_simplified_very_old     2.8502      2.900      0.983      0.328      -2.897       8.597
canton_Bern                      -11.8872      1.674     -7.102      0.000     -15.204      -8.570
canton_Luzern                     -3.9317      1.648     -2.385      0.019      -7.198      -0.665
==============================================================================
Omnibus:                       49.735   Durbin-Watson:                   1.692
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              134.776
Skew:                           1.588   Prob(JB):                     5.42e-30
Kurtosis:                       7.107   Cond. No.                         600.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Interpretation Guidelines
Because we set our baselines manually:

- All canton coefficients show the difference relative to Zurich
- All Bauperiode coefficients show the difference relative to “new (2011–2024)” buildings
- area_m2 indicates how price per m² changes with unit size

This allows clear insights into how building age and region affect rental value.